In [11]:
# %% [markdown]
# # LoRA Finetuning: Sarvam-1 on Itihasa Dataset
# Optimized for Apple Silicon (M4 Mac)

# %% [markdown]
# ## 1. Setup and Imports

# %%
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, TaskType

# Check for MPS availability
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# %% [markdown]
# ## 2. Configuration

# %%
# Model configuration
MODEL_NAME = "sarvamai/sarvam-1"
OUTPUT_DIR = "./sarvam1-itihasa-lora"

# Dataset subset configuration
TRAIN_SUBSET_SIZE = 20000  # Use first 20000 samples for training
VAL_SUBSET_SIZE = 200      # Use first 500 samples for validation

# Training configuration
MAX_SEQ_LENGTH = 256       # Sarvam-1 can handle longer sequences
NUM_EPOCHS = 1
BATCH_SIZE = 4           # Reduced for 2B model
GRADIENT_ACCUM_STEPS = 8   # Increased to compensate
LEARNING_RATE = 5e-6       # Lower LR for larger model

print(f"Training subset: {TRAIN_SUBSET_SIZE} samples")
print(f"Validation subset: {VAL_SUBSET_SIZE} samples")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM_STEPS}")

# %% [markdown]
# ## 3. Load Dataset from Local Parquet Files

# %%
# Load dataset from local parquet files
print("Loading Itihasa dataset from local parquet files...")
dataset = load_dataset(
    "parquet",
    data_files={
        "train": "ithasa/train/0000.parquet",
        "validation": "ithasa/val/0000.parquet",
        "test": "ithasa/test/0000.parquet"
    }
)

print(f"\nDataset structure:")
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")

# %% [markdown]
# ## 4. Explore Dataset

# %%
# Show sample data
print("\n" + "="*80)
print("Sample from dataset:")
print("="*80)
sample = dataset['train'][0]
print(f"\nFull sample structure: {sample.keys()}")
print(f"\nSample data:")
print(sample)

# %% [markdown]
# ## 5. Create Dataset Subsets

# %%
# Create subsets
print(f"\nCreating subsets...")
train_subset = dataset["train"].select(range(min(TRAIN_SUBSET_SIZE, len(dataset["train"]))))
val_subset = dataset["validation"].select(range(min(VAL_SUBSET_SIZE, len(dataset["validation"]))))

print(f"Train subset: {len(train_subset)} samples")
print(f"Validation subset: {len(val_subset)} samples")

# %% [markdown]
# ## 6. Load Tokenizer and Model

# %%
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded. Vocab size: {len(tokenizer)}")

# %%
# Load base model with 4-bit quantization for memory efficiency
print("Loading base model...")
print("⚠️ Note: Loading Sarvam-1 (2B parameters) - this may take a few minutes...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

# Move to MPS
model = model.to(device)

print(f"Model loaded. Parameters: {model.num_parameters():,}")

# %% [markdown]
# ## 7. Configure and Apply LoRA

# %%
# LoRA configuration - targeting more modules for better adaptation
print("Configuring LoRA...")
lora_config = LoraConfig(
    r=16,  # Slightly higher rank for 10B model
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Target all attention projections
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing for memory efficiency
model.enable_input_require_grads()

model.print_trainable_parameters()

# %% [markdown]
# ## 8. Preprocessing Function

# %%
def tokenize_fn(examples):
    """
    Tokenize with proper prompt masking for Sarvam-1.
    Only train on the output (English translation), not the input prompt (Sanskrit).
    """
    all_input_ids = []
    all_attention_masks = []
    all_labels = []
    
    for translation in examples['translation']:
        # Extract Sanskrit and English
        sn_text = str(translation['sn']).strip()
        en_text = str(translation['en']).strip()
        
        # Create prompt in a format suitable for Sarvam-1
        prompt = f"Translate this Sanskrit shloka to English:\n{sn_text}\n\nTranslation:"
        
        # Tokenize prompt separately to get its length
        prompt_tokens = tokenizer(prompt, add_special_tokens=True)
        prompt_len = len(prompt_tokens["input_ids"])
        
        # Create full sequence: prompt + output + eos
        full_text = prompt + " " + en_text + tokenizer.eos_token
        
        # Tokenize full sequence
        encoding = tokenizer(
            full_text,
            max_length=MAX_SEQ_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        input_ids = encoding["input_ids"][0].tolist()
        attention_mask = encoding["attention_mask"][0].tolist()
        
        # Create labels: mask prompt, keep output
        labels = input_ids.copy()
        
        # Mask the prompt tokens
        for i in range(min(prompt_len, len(labels))):
            labels[i] = -100
        
        # Mask padding tokens
        for i in range(len(labels)):
            if attention_mask[i] == 0:
                labels[i] = -100
        
        all_input_ids.append(input_ids)
        all_attention_masks.append(attention_mask)
        all_labels.append(labels)
    
    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_masks,
        "labels": all_labels
    }

# %% [markdown]
# ## 9. Tokenize Datasets

# %%
# Tokenize datasets
print("Tokenizing datasets...")
tokenized_train = train_subset.map(
    tokenize_fn,
    batched=True,
    batch_size=100,
    remove_columns=train_subset.column_names,
    desc="Tokenizing training data"
)

tokenized_val = val_subset.map(
    tokenize_fn,
    batched=True,
    batch_size=100,
    remove_columns=val_subset.column_names,
    desc="Tokenizing validation data"
)

print(f"✅ Tokenization complete")
print(f"Tokenized train samples: {len(tokenized_train)}")
print(f"Tokenized val samples: {len(tokenized_val)}")

# %% [markdown]
# ## 10. Debug Tokenization (Optional)

# %%
# Check a few examples
print("\n" + "="*80)
print("DEBUGGING TOKENIZED DATA")
print("="*80)

for i in range(min(3, len(tokenized_train))):
    example = tokenized_train[i]
    
    print(f"\n--- Example {i} ---")
    
    non_masked = sum(1 for label in example['labels'] if label != -100)
    masked = sum(1 for label in example['labels'] if label == -100)
    padding = sum(1 for mask in example['attention_mask'] if mask == 0)
    actual_tokens = len(example['input_ids']) - padding
    
    print(f"Sequence length: {len(example['input_ids'])}")
    print(f"Actual tokens (no padding): {actual_tokens}")
    print(f"Padding tokens: {padding}")
    print(f"Non-masked tokens (trainable): {non_masked}")
    print(f"Masked tokens: {masked}")
    print(f"Percentage trainable: {non_masked / len(example['labels']) * 100:.1f}%")
    
    if non_masked == 0:
        print("⚠️ WARNING: No trainable tokens!")

print("="*80)

# %% [markdown]
# ## 11. Setup Training

# %%
# Training arguments optimized for M4 Mac
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=100,
    eval_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=50,
    fp16=True,  # MPS supports fp16
    save_total_limit=2,
    dataloader_num_workers=0,  # Important for MPS
    remove_unused_columns=False,
    report_to="none",
    push_to_hub=False,
    max_grad_norm=1.0,
    gradient_checkpointing=True,  # Save memory
)

print("📝 Training Configuration:")
print(f"  - Model: {MODEL_NAME}")
print(f"  - Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Gradient accumulation: {GRADIENT_ACCUM_STEPS}")
print(f"  - Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM_STEPS}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Total epochs: {NUM_EPOCHS}")

# %%
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

print("Trainer initialized successfully!")

# %% [markdown]
# ## 12. Train the Model

# %%
# Start training
print("\n" + "="*80)
print("🚀 Starting training...")
print("="*80 + "\n")

trainer.train()

print("\n" + "="*80)
print("✅ Training complete!")
print("="*80)

# %% [markdown]
# ## 13. Save Model

# %%
# Save the final model
print(f"\nSaving model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved successfully!")
print(f"✓ Location: {OUTPUT_DIR}")

Using device: mps
Training subset: 20000 samples
Validation subset: 200 samples
Max sequence length: 256
Effective batch size: 32
Loading Itihasa dataset from local parquet files...

Dataset structure:
Train samples: 75162
Validation samples: 6149
Test samples: 11722

Sample from dataset:

Full sample structure: dict_keys(['translation'])

Sample data:
{'translation': {'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.', 'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}}

Creating subsets...
Train subset: 20000 samples
Validation subset: 200 samples
Loading tokenizer...
Tokenizer loaded. Vocab size: 68096
Loading base model...
⚠️ Note: Loading Sarvam-1 (2B parameters) - this may take a few minutes...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded. Parameters: 2,525,087,744
Configuring LoRA...
trainable params: 6,422,528 || all params: 2,531,510,272 || trainable%: 0.2537
Tokenizing datasets...


Tokenizing training data:   0%|          | 0/20000 [00:00<?, ? examples/s]

Tokenizing validation data:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Tokenization complete
Tokenized train samples: 20000
Tokenized val samples: 200

DEBUGGING TOKENIZED DATA

--- Example 0 ---
Sequence length: 256
Actual tokens (no padding): 112
Padding tokens: 144
Non-masked tokens (trainable): 112
Masked tokens: 144
Percentage trainable: 43.8%

--- Example 1 ---
Sequence length: 256
Actual tokens (no padding): 97
Padding tokens: 159
Non-masked tokens (trainable): 97
Masked tokens: 159
Percentage trainable: 37.9%

--- Example 2 ---
Sequence length: 256
Actual tokens (no padding): 95
Padding tokens: 161
Non-masked tokens (trainable): 95
Masked tokens: 161
Percentage trainable: 37.1%
📝 Training Configuration:
  - Model: sarvamai/sarvam-1
  - Max sequence length: 256
  - Batch size: 4
  - Gradient accumulation: 8
  - Effective batch size: 32
  - Learning rate: 5e-06
  - Total epochs: 1
Trainer initialized successfully!

🚀 Starting training...



/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss
100,3.014800,2.909653
200,2.638900,2.585361
300,2.521800,2.515157
400,2.480700,2.484417
500,2.463100,2.469894
600,2.462200,2.464713


/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true 


✅ Training complete!

Saving model to ./sarvam1-itihasa-lora...
✅ Model saved successfully!
✓ Location: ./sarvam1-itihasa-lora


In [12]:
# =========================================================
# Sarvam-1 3B + LoRA — CPU ONLY (STABLE, NO KERNEL DEATH)
# =========================================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# -----------------------------
# Force CPU everywhere
# -----------------------------
device = torch.device("cpu")

# -----------------------------
# Load tokenizer
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")

# -----------------------------
# Load base model (CPU, float16)
# -----------------------------
base_model = AutoModelForCausalLM.from_pretrained(
    "sarvamai/sarvam-1",
    torch_dtype=torch.float16,
    device_map={"": "cpu"}
)

# -----------------------------
# Load LoRA adapter (CPU)
# -----------------------------
model = PeftModel.from_pretrained(
    base_model,
    "./sarvam1-itihasa-lora/",
    device_map={"": "cpu"}
)

model.eval()

# -----------------------------
# Inference
# -----------------------------

# Test samples
test_samples = [
    "धर्मो रक्षति रक्षितः",
    "सत्यमेव जयते",
    "वसुधैव कुटुम्बकम्",
    "अहिंसा परमो धर्मः",
]

prompt = f"translate in sanskrit: {test_samples[0]}"
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


translate in sanskrit: धर्मो रक्षति रक्षितः ।
न तर्हि रक्षसा रक्ष्यते रक्षसः ।
रक्षार्थं हि तत् रक्षत रक्षसां रक्षितारः ॥



In [13]:
prompt = f"translate in sanskrit: {test_samples[3]}"
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


translate in sanskrit: अहिंसा परमो धर्मः
अहिंसा परमं धर्मः (a-himsā paro dharmaḥ) literally means "Ahimsa is the highest religion". This phrase appears several times in the Tripitaka, especially in the Mahavagga and the Parivrâjakavagga of the Suttanipata. The first verse quoted here also uses the word Dharma as an alternative to 'ahimsa'. However,


In [34]:
num_samples = 20

for i in range(min(num_samples, len(val_subset))):
    sample = val_subset[i]

    sanskrit_text = sample["translation"]["sn"]
    expected_en = sample["translation"]["en"]

    prompt = (
        "Translate this Sanskrit shloka to English:\n"
        f"{sanskrit_text}\n\n"
        "Translation:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Translation:" in decoded:
        generated_en = decoded.split("Translation:")[-1].strip()
    else:
        generated_en = decoded.strip()

    print("=" * 100)
    print(f"Sample {i + 1}")
    print("-" * 100)

    print("📜 Sanskrit:")
    print(sanskrit_text)

    print("\n🎯 Expected:")
    print(expected_en)

    print("\n✨ Model Output:")
    print(generated_en)

    print()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sample 1
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
तस्यां चीरं वसानायां नाथवत्यामनाथवत्। प्रचुक्रोश जनः सर्वो धिक् त्वां दशरथं त्विति ॥

🎯 Expected:
When Şītā, having a husband although seeming as if she had none, was putting on the ascetic guise, the people got into a wrath and exclaimed, “O Dasaratha, fie on you!"

✨ Model Output:
The people, enraged by his arrogance, exclaimed, O Lord of men! This Nala is worse than your wife, O King.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sample 2
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
तेन तत्र प्रणादेन दुःखितः स महीपतिः। चिच्छेद जीविते श्रद्धां धर्मे यशसि चात्मनः॥ स निःश्वस्योष्णमैक्ष्वाकस्तां भार्यामिदमब्रवीत्। कैकेयि कुशचीरेण न सीता गन्तुमर्हति॥

🎯 Expected:
Aggrieved at the uproar that arose there in consequence, the lord of earth banished from his heart all regard for life, virtue, and fame. And sighing hot, that descendant of Ikşvāku spoke to that wife of his, saying, O Kaikeyi, Sītā deserves not to go in a Kuća dress.

✨ Model Output:
Thus, the king being saddened at that time and with disgusted mind thought within himself, "How can I go on living in such a way? How is it possible for me to be able to meet my wife Ceyté here. (He) said unto his queen Ceyté, "Come not to see us now."



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Sample 17
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
सोऽहं पितुर्वचः श्रुत्वा शासनाद् ब्रह्मवादिनः। करिष्यामि न संदेहस्ताटकावधमुत्तमम्॥

🎯 Expected:
Therefore, commanded by that upholder of the Veda, I, agreeably to my father's mandate, will, without doubt, bring about that welcome event-the death of Tataka.

✨ Model Output:
I, having heard the words of my father, will be performing a sacrifice in which I shall slay him.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sample 18
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
गोब्राह्मणहितार्थाय देशस्य च हिताय च। तव चैवाप्रमेयस्य वचनं कर्तुमुद्यतः॥

🎯 Expected:
And in the interests of Brāhmaṇas, kine, and celestials, I am ready to act as desired by you of immeasurable energy.

✨ Model Output:
For the welfare of the Brahmanas and the interest of country also, I have said it.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sample 19
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
एवमुक्त्वा धनुर्मध्ये बद्ध्वा मुष्टिमरिंदमः। ज्याघोषमकरोत् तीव्र दिशः शब्देन नादयन् ॥ तेन शब्देन विवस्तास्ताटकावनवासिनः। ताटका च सुसंक्रुद्धा तेन शब्देन मोहिता॥

🎯 Expected:
Having said this, that repressor of foes, with clenched fist, twanged his bow-string, filling the ten cardinal points with the sounds. And at those sounds, the dwellers in Tāțakā's forest were filled with perturbation, and Tātakā also amazed at those sounds, became exceedingly wroth.

✨ Model Output:
So saying, he bound his bow in the middle and hurled a mace at the sky. The voice of thunder sounding from afar filled with wonderment all those inside the tataka (town).

Sample 20
----------------------------------------------------------------------------------------------------
📜 Sanskrit:
तं शब्दमभिनिध्याय राक्षसी क्रोधमूर्छिता। श्रुत्वा चाभ्यद्रवत् क्रुद्धा यत्र शब्दो विनिःसृतः॥

🎯 Expected:
